## Import Libraries

In [25]:
import pandas as pd
import numpy as np

import re

import nltk
nltk.download('stopwords')      
nltk.download('punkt')  
nltk.download('punkt_tab')         
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('indonesian'))



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Abiyoso\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Abiyoso\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Abiyoso\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Abiyoso\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Load Data

In [6]:
tweet_df = pd.read_csv('data.csv', encoding='ISO-8859-1')
tweet_df.head()

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong
0,- disaat semua cowok berusaha melacak perhatia...,1,1,1,0,0,0,0,0,1,1,0,0
1,RT USER: USER siapa yang telat ngasih tau elu?...,0,1,0,0,0,0,0,0,0,0,0,0
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0,0,0,0,0,0,0,0,0,0,0,0
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0,0,0,0,0,0,0,0,0,0,0,0
4,USER USER Kaum cebong kapir udah keliatan dong...,1,1,0,1,1,0,0,0,0,0,1,0


### Contoh tweet Non Hate-speech

In [8]:
tweet_df[tweet_df['HS']== 0].head()

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong
1,RT USER: USER siapa yang telat ngasih tau elu?...,0,1,0,0,0,0,0,0,0,0,0,0
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0,0,0,0,0,0,0,0,0,0,0,0
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0,0,0,0,0,0,0,0,0,0,0,0
6,deklarasi pilkada 2018 aman dan anti hoax warg...,0,0,0,0,0,0,0,0,0,0,0,0
7,Gue baru aja kelar re-watch Aldnoah Zero!!! pa...,0,1,0,0,0,0,0,0,0,0,0,0


### Contoh tweet HS

In [9]:
tweet_df[tweet_df['HS']==1].head()

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong
0,- disaat semua cowok berusaha melacak perhatia...,1,1,1,0,0,0,0,0,1,1,0,0
4,USER USER Kaum cebong kapir udah keliatan dong...,1,1,0,1,1,0,0,0,0,0,1,0
5,USER Ya bani taplak dkk \xf0\x9f\x98\x84\xf0\x...,1,1,0,1,0,0,0,0,1,0,1,0
10,"Setidaknya gw punya jari tengah buat lu, sebel...",1,1,1,0,0,0,0,0,1,1,0,0
11,USER USER USER USER BANCI KALENG MALU GA BISA ...,1,1,1,0,0,0,0,1,0,1,0,0


### Check missing values

In [11]:
tweet_df.isnull().sum().sort_values(ascending=False)

Tweet            0
HS               0
Abusive          0
HS_Individual    0
HS_Group         0
HS_Religion      0
HS_Race          0
HS_Physical      0
HS_Gender        0
HS_Other         0
HS_Weak          0
HS_Moderate      0
HS_Strong        0
dtype: int64

### Function to process the data

In [35]:
def data_processing(tweet):
    tweet = re.sub(r"https\S+|www\S+http\S+", '', tweet, flags = re.MULTILINE)
    tweet = re.sub(r'\@w+|\#','', tweet)
    tweet = re.sub(r'[^\w\s]','',tweet)
    tweet = re.sub('RT', '', tweet)
    tweet = re.sub(r'ð','',tweet)
    tweet = re.sub("[^a-zA-Z^']", " ", tweet)
    tweet = re.sub(" {2,}", " ", tweet)
    filtered_tweets = [w for w in tweet_tokens if not w in stop_words]
    tweet = tweet.lower()
    tweet_tokens = word_tokenize(tweet)
    return " ".join(filtered_tweets)

In [36]:
tweet_df.Tweet = tweet_df['Tweet'].apply(data_processing)

UnboundLocalError: cannot access local variable 'tweet_tokens' where it is not associated with a value

In [27]:
tweet_df = tweet_df.drop_duplicates('Tweet')

In [28]:
lemmatizer = WordNetLemmatizer()
def lemmatizing(data):
    tweet = [lemmatizer.lemmatize(word) for word in data]
    return data

In [29]:
tweet_df['Tweet'] = tweet_df['Tweet'].apply(lambda x: lemmatizing(x))

In [30]:
print(tweet_df['Tweet'].iloc[0],"\n")
print(tweet_df['Tweet'].iloc[1],"\n")
print(tweet_df['Tweet'].iloc[2],"\n")
print(tweet_df['Tweet'].iloc[3],"\n")
print(tweet_df['Tweet'].iloc[4],"\n")

disaat cowok berusaha melacak perhatian gue loe lantas remehkan perhatian yg gue kasih khusus elo basic elo cowok bego 

rt user user telat ngasih tau eluedan sarap gue bergaul cigax jifla calis noh licew 

41 kadang berfikir percaya tuhan jatuh berkalikali kadang tuhan ninggalkan orangtuaku berencana berpisah kakakku memilih kristen anak ter 

user user akunnku tau matamu sipit diliat 

user user kaum cebong kapir udah keliatan dongoknya dongok hahahah 

